In [47]:
import statsapi as mlb
import pandas as pd
import sqlalchemy
import config
import csv
from os import walk
import re
from datetime import datetime

In [84]:
mlb.ENDPOINTS.keys()

dict_keys(['attendance', 'awards', 'conferences', 'divisions', 'draft', 'game', 'game_diff', 'game_timestamps', 'game_changes', 'game_contextMetrics', 'game_winProbability', 'game_boxscore', 'game_content', 'game_color', 'game_color_diff', 'game_color_timestamps', 'game_linescore', 'game_playByPlay', 'gamePace', 'highLow', 'homeRunDerby', 'league', 'league_allStarBallot', 'league_allStarWriteIns', 'league_allStarFinalVote', 'people', 'people_changes', 'people_freeAgents', 'person', 'person_stats', 'jobs', 'jobs_umpires', 'jobs_umpire_games', 'jobs_datacasters', 'jobs_officialScorers', 'schedule', 'schedule_tied', 'schedule_postseason', 'schedule_postseason_series', 'schedule_postseason_tuneIn', 'seasons', 'season', 'sports', 'sports_players', 'standings', 'stats', 'stats_leaders', 'stats_streaks', 'teams', 'teams_history', 'teams_stats', 'teams_affiliates', 'team', 'team_alumni', 'team_coaches', 'team_personnel', 'team_leaders', 'team_roster', 'venue', 'meta'])

In [287]:
mlb.ENDPOINTS['meta']

{'url': 'https://statsapi.mlb.com/api/{ver}/{type}',
 'path_params': {'ver': {'type': 'str',
   'default': 'v1',
   'leading_slash': False,
   'trailing_slash': False,
   'required': True},
  'type': {'type': 'str',
   'default': None,
   'leading_slash': False,
   'trailing_slash': False,
   'required': True}},
 'query_params': [[]],
 'required_params': [[]],
 'note': 'The meta endpoint is used to retrieve values to be used within other API calls. Available types: awards, baseballStats, eventTypes, gameStatus, gameTypes, hitTrajectories, jobTypes, languages, leagueLeaderTypes, logicalEvents, metrics, pitchCodes, pitchTypes, platforms, positions, reviewReasons, rosterTypes, scheduleEventTypes, situationCodes, sky, standingsTypes, statGroups, statTypes, windDirection.'}

In [306]:
mlb.get('meta',{'type':'rosterTypes'})

[{'description': '40 man roster for a team',
  'lookupName': '40man',
  'parameter': '40Man'},
 {'description': 'Full roster including active and inactive players for a season',
  'lookupName': 'fullSeason',
  'parameter': 'fullSeason'},
 {'description': 'Full roster including active and inactive players',
  'lookupName': 'full',
  'parameter': 'fullRoster'},
 {'description': 'Non-Roster Invitees',
  'lookupName': 'nonRosterInvitees',
  'parameter': 'nonRosterInvitees'},
 {'description': 'Active roster for a team',
  'lookupName': 'active',
  'parameter': 'active'},
 {'description': 'All Time roster for a team',
  'lookupName': 'alltime',
  'parameter': 'allTime'},
 {'description': 'Depth chart for a team',
  'lookupName': 'active',
  'parameter': 'depthChart'},
 {'description': 'Roster for day of game',
  'lookupName': 'active',
  'parameter': 'gameday'},
 {'description': 'Coach roster for a team',
  'lookupName': 'active',
  'parameter': 'coach'}]

In [164]:
def get_gamePks(seasons):
    """
    Returns a list of gamePks for non-exhibition games given a list of seasons as integers.
    """
    import statsapi as mlb
    import csv
    import time
    import sys
    
    gamePks_path = "/Users/schlinkertc/code/mlb_predictions/gamePks"
    
    from os import walk
    import re
    
    # walk the gamePks directory to find the seasons that we've already added 
    f = []
    for (dirpath, dirnames, filenames) in walk(gamePks_path):
        f.extend(filenames)
        break
    years = [re.findall('[^.csv]+',x) for x in f]
    already_added = [int(item) for sublist in years for item in sublist if item[0] in ['1','2']]
    
    seasons = list(set(seasons)-set(already_added))
    
    gamePks = {}
    for season in seasons:
        mlb.get('season',{'sportId':1,'seasonId':str(season)})
        try:
            games = mlb.schedule(start_date=f'02/01/{season}',
                                    end_date=f'11/30/{season}',
                                    sportId=1)
            pks = [x['game_id'] for x in games]
            print(pks[0])
            
            gamePks[season]=pks
            print(len(gamePks))
            with open(gamePks_path + f'/{season}.csv', 'w',newline='') as myfile:
                wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
                wr.writerow(gamePks[season])
        except ValueError as err:
            print(f'{season} failed. Error: {err} Waiting 10 seconds before resuming')
            time.sleep(10)
            #seasons.append(season)
        except:
            print("Unexpected error:", sys.exc_info()[0])
            raise
            
        
    return gamePks   

In [165]:
seasons = list(range(2009,2020))

In [167]:
pks = get_gamePks(seasons)

In [175]:
gamePks_path = "/Users/schlinkertc/code/mlb_predictions/gamePks"
f = []
for (dirpath, dirnames, filenames) in walk(gamePks_path):
    f.extend(filenames)
    break
pk_paths = [gamePks_path + '/' + x for x in f if x[0]!= '.']
pk_paths

['/Users/schlinkertc/code/mlb_predictions/gamePks/2009.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2019.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2018.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2015.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2014.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2016.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2017.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2013.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2012.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2010.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2011.csv']

In [189]:
seasons = [item for sublist in [re.findall('/gamePks/([^.csv]+)',x) for x in pk_paths] for item in sublist]

In [191]:
season = re.findall('/gamePks/([^.csv]+)',pk_paths[0])

In [192]:
season

['2009']

In [176]:

gamePks = {}
with open(pk_paths[0], 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)

In [4]:
def read_gamePks():
    gamePks_path = "/Users/schlinkertc/code/mlb_predictions/gamePks"
    f = []
    for (dirpath, dirnames, filenames) in walk(gamePks_path):
        f.extend(filenames)
        break
    pk_paths = [gamePks_path + '/' + x for x in f if x[0]!= '.']
    
    gamePks = {}
    for path in pk_paths:
        season = re.findall('/gamePks/([^.csv]+)',path)
        with open(path, 'r') as f:
            reader = csv.reader(f)
            seasonPks = list(reader)
        gamePks[season[0]] = [item for sublist in seasonPks for item in sublist]
    return gamePks

In [29]:
gamePk_dict = read_gamePks()

In [30]:
test_gamePk = gamePk_dict['2019'][1000]

In [86]:
game_test = mlb.get('game',{'gamePk':566651})

In [87]:
game_test.keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

In [88]:
game_test['liveData']['plays']

{'allPlays': [{'result': {'type': 'atBat',
    'event': 'Groundout',
    'eventType': 'field_out',
    'description': 'Ehire Adrianza grounds out, third baseman Matt Duffy to first baseman Yandy Diaz.',
    'rbi': 0,
    'awayScore': 0,
    'homeScore': 0},
   'about': {'atBatIndex': 0,
    'halfInning': 'top',
    'inning': 1,
    'startTime': '1970-01-01T00:00:00.000Z',
    'endTime': '2019-02-28T23:37:05.000Z',
    'isComplete': True,
    'isScoringPlay': False,
    'hasReview': False,
    'hasOut': True,
    'captivatingIndex': 0},
   'count': {'balls': 0, 'strikes': 0, 'outs': 1},
   'matchup': {'batter': {'id': 501303,
     'fullName': 'Ehire Adrianza',
     'link': '/api/v1/people/501303'},
    'batSide': {'code': 'R', 'description': 'Right'},
    'pitcher': {'id': 605483,
     'fullName': 'Blake Snell',
     'link': '/api/v1/people/605483'},
    'pitchHand': {'code': 'L', 'description': 'Left'},
    'batterHotColdZones': [],
    'pitcherHotColdZones': [],
    'splits': {'batter

In [33]:
[x for x in game_test['gameData'].keys()]

['game',
 'datetime',
 'status',
 'teams',
 'players',
 'venue',
 'weather',
 'review',
 'flags',
 'alerts',
 'probablePitchers',
 'officialScorer',
 'primaryDatacaster']

In [34]:
game_test['gameData']['game']

{'pk': 566781,
 'type': 'R',
 'doubleHeader': 'N',
 'id': '2019/05/04/tormlb-texmlb-1',
 'gamedayType': 'P',
 'tiebreaker': 'N',
 'gameNumber': 1,
 'calendarEventID': '14-566781-2019-05-04',
 'season': '2019',
 'seasonDisplay': '2019'}

In [35]:
game_test['gameData']['datetime']

{'dateTime': '2019-05-05T00:05:00Z',
 'originalDate': '2019-05-04',
 'dayNight': 'night',
 'time': '7:05',
 'ampm': 'PM'}

In [69]:
test_date = game_test['gameData']['datetime']['dateTime']

In [70]:
datetime.strptime(test_date,datetime.strptime(test_date,'%Y-%m-%dT%H:%M:%SZ'))

datetime.datetime(2019, 5, 5, 0, 5)

In [62]:
test_time = game_test['gameData']['datetime']['time']+game_test['gameData']['datetime']['ampm']
test_time

'7:05PM'

In [66]:
import time

In [74]:
datetime.strptime(test_time,"%H:%M%p").time()

datetime.time(7, 5)

In [36]:
game_test['gameData']['status']

{'abstractGameState': 'Final',
 'codedGameState': 'F',
 'detailedState': 'Final',
 'statusCode': 'F',
 'abstractGameCode': 'F'}

In [90]:
game_test['gameData']['teams']['home']['id']

139

In [91]:
game_test['gameData']['teams']['away']['id']

142

In [92]:
gameData = game_test['gameData']

In [93]:
gameData['venue']

{'id': 2534,
 'name': 'Charlotte Sports Park',
 'link': '/api/v1/venues/2534',
 'location': {'city': 'Port Charlotte',
  'state': 'Florida',
  'stateAbbrev': 'FL',
  'defaultCoordinates': {'latitude': 26.99888, 'longitude': -82.18212}},
 'timeZone': {'id': 'America/New_York', 'offset': -5, 'tz': 'EST'},
 'fieldInfo': {'capacity': 6000,
  'turfType': 'Grass',
  'roofType': 'Open',
  'leftLine': 343,
  'center': 414,
  'rightLine': 343}}

In [94]:
gameData['probablePitchers']

{'away': {'id': 606167,
  'fullName': 'Mejia, Adalberto',
  'link': '/api/v1/people/606167'},
 'home': {'id': 605483,
  'fullName': 'Snell, Blake',
  'link': '/api/v1/people/605483'}}

In [96]:
game_test.keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

## plays

In [254]:
play_test = game_test['liveData']['plays']['allPlays'][25]

In [269]:
play_test.keys()

dict_keys(['result', 'about', 'count', 'matchup', 'pitchIndex', 'actionIndex', 'runnerIndex', 'runners', 'playEvents', 'atBatIndex', 'playEndTime'])

In [255]:
play_test['result']

{'type': 'atBat',
 'event': 'Grounded Into DP',
 'eventType': 'grounded_into_double_play',
 'description': 'Jake Cave grounds into a double play, pitcher Colin Poche to second baseman Brandon Lowe to first baseman Yandy Diaz.   C.J. Cron out at 2nd.    Jake Cave out at 1st.',
 'rbi': 0,
 'awayScore': 0,
 'homeScore': 0}

In [256]:
play_test['about']

{'atBatIndex': 25,
 'halfInning': 'top',
 'inning': 4,
 'startTime': '2019-03-01T00:43:41.000Z',
 'endTime': '2019-03-01T00:45:54.000Z',
 'isComplete': True,
 'isScoringPlay': False,
 'hasReview': False,
 'hasOut': True,
 'captivatingIndex': 24}

In [257]:
play_test['matchup']

{'batter': {'id': 595909,
  'fullName': 'Jake Cave',
  'link': '/api/v1/people/595909'},
 'batSide': {'code': 'L', 'description': 'Left'},
 'pitcher': {'id': 621363,
  'fullName': 'Colin Poche',
  'link': '/api/v1/people/621363'},
 'pitchHand': {'code': 'L', 'description': 'Left'},
 'batterHotColdZones': [],
 'pitcherHotColdZones': [],
 'splits': {'batter': 'vs_LHP', 'pitcher': 'vs_LHB', 'menOnBase': 'Empty'}}

In [258]:
play_test['pitchIndex']

[1]

In [259]:
play_test['runnerIndex']

[0, 1]

In [260]:
play_test['actionIndex']

[0]

In [261]:
play_test['count']

{'balls': 0, 'strikes': 0, 'outs': 3}

In [267]:
game_test['gameData']['game']

{'pk': 566651,
 'type': 'S',
 'doubleHeader': 'N',
 'id': '2019/02/28/minmlb-tbamlb-1',
 'gamedayType': 'Y',
 'tiebreaker': 'N',
 'gameNumber': 1,
 'calendarEventID': '14-566651-2019-02-28',
 'season': '2019',
 'seasonDisplay': '2019'}

In [ ]:
plays_test = game_test['liveData']['plays']['allPlays'][x]

In [ ]:
def get_pitches(api_call):
    """
    Takes in the result of of mlbstats-api call on the game endpoint. 
    Returns list of dictionaries for each play
    """
    gameData = api_call['gameData']
    game = gameData['game']
    
    liveData = api_call['liveData']
    
    

## pitches

In [262]:
[play_test['playEvents'][x] for x in play_test['pitchIndex']]

[{'details': {'call': {'code': 'X', 'description': 'Hit Into Play - Out(s)'},
   'description': 'In play, out(s)',
   'code': 'X',
   'ballColor': 'rgba(26, 86, 190, 1.0)',
   'isInPlay': True,
   'isStrike': False,
   'isBall': False,
   'hasReview': False},
  'count': {'balls': 0, 'strikes': 0},
  'pitchData': {'strikeZoneTop': 3.359,
   'strikeZoneBottom': 1.7,
   'coordinates': {'x': 107.45, 'y': 172.0},
   'breaks': {}},
  'hitData': {'trajectory': 'ground_ball',
   'hardness': 'medium',
   'location': '1',
   'coordinates': {'coordX': 137.41, 'coordY': 156.27}},
  'index': 1,
  'playId': '05666516-0266-0013-000c-f08cd117d70a',
  'pitchNumber': 1,
  'startTime': '2019-03-01T00:45:36.000Z',
  'endTime': '2019-03-01T00:45:54.000Z',
  'isPitch': True,
  'type': 'pitch'}]

## Actions 

In [263]:
[play_test['playEvents'][x] for x in play_test['actionIndex']]

[{'details': {'description': 'Mound Visit.',
   'event': 'Game Advisory',
   'awayScore': 0,
   'homeScore': 0,
   'isScoringPlay': False,
   'hasReview': False},
  'count': {'balls': 0, 'strikes': 0, 'outs': 1},
  'index': 0,
  'startTime': '2019-03-01T00:43:46.000Z',
  'endTime': '2019-03-01T00:45:36.000Z',
  'isPitch': False,
  'type': 'action',
  'player': {'id': 595909, 'link': '/api/v1/people/595909'}}]

## Runners

In [273]:
len(play_test['runners'])

2

In [282]:
play_test['runners'][0]['movement']['start']

'1B'

In [283]:
[play_test['runners'][x]['movement']['start'] for x in play_test['runnerIndex']]

['1B', None]

## Teams

In [361]:
mlb.ENDPOINTS['team']

{'url': 'https://statsapi.mlb.com/api/{ver}/teams/{teamId}',
 'path_params': {'ver': {'type': 'str',
   'default': 'v1',
   'leading_slash': False,
   'trailing_slash': False,
   'required': True},
  'teamId': {'type': 'str',
   'default': None,
   'leading_slash': False,
   'trailing_slash': False,
   'required': True}},
 'query_params': ['season', 'sportId', 'hydrate', 'fields'],
 'required_params': [[]]}

In [364]:
team_test = mlb.get('team',{'teamId':120})

In [370]:
team_test=team_test['teams']

TypeError: list indices must be integers or slices, not str

In [371]:
team_test[0]

{'id': 120,
 'name': 'Washington Nationals',
 'link': '/api/v1/teams/120',
 'venue': {'id': 3309,
  'name': 'Nationals Park',
  'link': '/api/v1/venues/3309'},
 'teamCode': 'was',
 'fileCode': 'was',
 'abbreviation': 'WSH',
 'teamName': 'Nationals',
 'locationName': 'Washington',
 'firstYearOfPlay': '1969',
 'league': {'id': 104,
  'name': 'National League',
  'link': '/api/v1/league/104'},
 'division': {'id': 204,
  'name': 'National League East',
  'link': '/api/v1/divisions/204'},
 'sport': {'id': 1,
  'link': '/api/v1/sports/1',
  'name': 'Major League Baseball'},
 'shortName': 'Washington',
 'springLeague': {'id': 115,
  'name': 'Grapefruit League',
  'link': '/api/v1/league/115',
  'abbreviation': 'GL'},
 'allStarStatus': 'N',
 'active': True}

## Roster

In [300]:
mlb.ENDPOINTS.keys()

dict_keys(['attendance', 'awards', 'conferences', 'divisions', 'draft', 'game', 'game_diff', 'game_timestamps', 'game_changes', 'game_contextMetrics', 'game_winProbability', 'game_boxscore', 'game_content', 'game_color', 'game_color_diff', 'game_color_timestamps', 'game_linescore', 'game_playByPlay', 'gamePace', 'highLow', 'homeRunDerby', 'league', 'league_allStarBallot', 'league_allStarWriteIns', 'league_allStarFinalVote', 'people', 'people_changes', 'people_freeAgents', 'person', 'person_stats', 'jobs', 'jobs_umpires', 'jobs_umpire_games', 'jobs_datacasters', 'jobs_officialScorers', 'schedule', 'schedule_tied', 'schedule_postseason', 'schedule_postseason_series', 'schedule_postseason_tuneIn', 'seasons', 'season', 'sports', 'sports_players', 'standings', 'stats', 'stats_leaders', 'stats_streaks', 'teams', 'teams_history', 'teams_stats', 'teams_affiliates', 'team', 'team_alumni', 'team_coaches', 'team_personnel', 'team_leaders', 'team_roster', 'venue', 'meta'])

In [304]:
mlb.ENDPOINTS['team_roster']

{'url': 'https://statsapi.mlb.com/api/{ver}/teams/{teamId}/roster',
 'path_params': {'ver': {'type': 'str',
   'default': 'v1',
   'leading_slash': False,
   'trailing_slash': False,
   'required': True},
  'teamId': {'type': 'str',
   'default': None,
   'leading_slash': False,
   'trailing_slash': False,
   'required': True}},
 'query_params': ['rosterType', 'season', 'date', 'hydrate', 'fields'],
 'required_params': [['rosterType', 'season']]}

In [356]:
roster_test = mlb.get('team_roster',{'teamId':146,'rosterType':'active','season':1995,})

In [357]:
roster_test

{'copyright': 'Copyright 2020 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'roster': [{'person': {'id': 110011,
    'fullName': 'Kurt Abbott',
    'link': '/api/v1/people/110011'},
   'jerseyNumber': '',
   'position': {'code': '6',
    'name': 'Shortstop',
    'type': 'Infielder',
    'abbreviation': 'SS'},
   'status': {'code': 'A', 'description': 'Active'}},
  {'person': {'id': 110312,
    'fullName': 'Alex Arias',
    'link': '/api/v1/people/110312'},
   'jerseyNumber': '',
   'position': {'code': '5',
    'name': 'Third Base',
    'type': 'Infielder',
    'abbreviation': '3B'},
   'status': {'code': 'RL', 'description': 'Released'}},
  {'person': {'id': 110536,
    'fullName': 'Willie Banks',
    'link': '/api/v1/people/110536'},
   'jerseyNumber': '',
   'position': {'code': '1',
    'name': 'Pitcher',
    'type': 'Pitcher',
    'abbreviation': 'P'},
   'status': {'code': '

In [303]:
mlb.get('game_contextMetrics',{'gamePk':test_gamePk})

{'game': {'gamePk': 566781,
  'link': '/api/v1/game/566781/feed/live',
  'gameType': 'R',
  'season': '2019',
  'gameDate': '2019-05-05T00:05:00Z',
  'status': {'abstractGameState': 'Final',
   'codedGameState': 'F',
   'detailedState': 'Final',
   'statusCode': 'F',
   'abstractGameCode': 'F'},
  'teams': {'away': {'leagueRecord': {'wins': 15, 'losses': 18, 'pct': '.455'},
    'score': 5,
    'team': {'id': 141,
     'name': 'Toronto Blue Jays',
     'link': '/api/v1/teams/141'},
    'isWinner': False,
    'probablePitcher': {'id': 623381,
     'fullName': 'Pannone, Thomas',
     'link': '/api/v1/people/623381'},
    'splitSquad': False,
    'seriesNumber': 11},
   'home': {'leagueRecord': {'wins': 15, 'losses': 16, 'pct': '.484'},
    'score': 8,
    'team': {'id': 140, 'name': 'Texas Rangers', 'link': '/api/v1/teams/140'},
    'isWinner': True,
    'probablePitcher': {'id': 458681,
     'fullName': 'Lynn, Lance',
     'link': '/api/v1/people/458681'},
    'splitSquad': False,
    's